In [1]:
import os
import pydicom
import shutil
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pandas as pd
from tqdm import tqdm
import os
import os.path as osp
from glob import glob
import json
from collections import defaultdict
import skimage
from multiprocessing import Pool 
import math
import random

HOSPITALS = {
    "KG" : "KUMC (고려대학교병원)",
    "CA" : "CAUMC (중앙대학교병원)",
    "CB" : "CBNUH (충북대학교병원)",
    "CN" : "CNUH (충남대학교병원)",
    "DA" : "DAMC (동아대학교병원)",
    "DE" : "DEMC (동의의료원)",
    "DI" : "DUMC (동국대학교병원)",
    "GN" : "JNUH (제주대학교병원)",
    "HS" : "HUMC (한림대학교병원)",
    "KMD" : "DSMC (계명대학교병원)",
    "SH" : "SCHMC (순천향대학교병원)",
    "SM" : "seoulMC (서울의료원)",
    "YN" : "YUMC (영남대학교병원)" 
}

HOSPITALS_ENG = {
    "KG" : "KUMC",
    "CA" : "CAUMC",
    "CB" : "CBNUH",
    "CN" : "CNUH",
    "DA" : "DAMC",
    "DE" : "DEMC",
    "DI" : "DUMC",
    "GN" : "JNUH",
    "HS" : "HUMC",
    "KMD" : "DSMC",
    "SH" : "SCHMC",
    "SM" : "seoulMC",
    "YN" : "YUMC" 
}

In [ ]:
def read_dicom(dcm_path):
    dcm = pydicom.dcmread(dcm_path,force=True)
    p
    required_elements = ['PixelData', 'BitsAllocated', 'Rows', 'Columns',
                     'PixelRepresentation', 'SamplesPerPixel','PhotometricInterpretation',
                        'BitsStored','HighBit']
    missing = [elem for elem in required_elements if elem not in dcm]
    for elem in required_elements:
        if elem not in dcm:
            if elem== 'BitsAllocated':
                dcm.BitsAllocated = 16
            if elem== 'PixelRepresentation':
                dcm.PixelRepresentation = 1
            if elem== 'SamplesPerPixel':
                dcm.SamplesPerPixel = 1
            if elem== 'PhotometricInterpretation':
                dcm.PhotometricInterpretation = 'MONOCHROME2'
            if elem== 'BitsStored':
                dcm.BitsStored = 12
            if elem== 'BitsStored':
                dcm.BitsStored = 11
    return dcm

def cmb_count(arr):
    """
    Args:
    - arr = Label array (either y_pred or y_true)
    Return:
    - cmb_cnt = number of microbleeds counted
    """
    if np.sum(arr) == 0:
        return 0
    arr = np.squeeze(arr)
    labeled_image = skimage.measure.label(arr)
    return int(np.max(labeled_image))

In [63]:
# Files download from MEDIHUB have prefix (xxxxx). Remove it
for folder in tqdm(glob("/data000/mhchoi/CMB/SWI_annotation_new/*")):
    if osp.basename(folder)[1:6].isdigit(): # (63886)KMD-09862-1 => KMD-09862-1
        try:
            os.rename(folder, osp.join(osp.dirname(folder), osp.basename(folder)[7:]))
        except OSError as e:
            print(e)

100%|██████████| 711/711 [00:00<00:00, 983883.25it/s]


In [ ]:
# CRCKS annotation (0,1,2) => AISCAN annotation (0,1) : Set all 'possible CMBs' to CMBs
exist = set()
for nii_path in tqdm(glob("/data000/jhpark/CMB/SWI_annotation/*/*")): 
    folder = osp.dirname(nii_path)
    patient_id = osp.basename(folder)
    if patient_id in exist:
        print(patient_id, "has more than one nii file. Skip it")
        continue
    exist.add(patient_id)
    nii_array = np.array(nib.load(nii_path).dataobj)
    nii_array[nii_array == 2] = 1 # possible CMB => CMB
    os.makedirs(osp.join("/data000/mhchoi/CMB/SWI_annotation_old", patient_id), exist_ok=True)
    nib.save(nib.Nifti1Image(nii_array, np.eye(4)), osp.join("/data000/mhchoi/CMB/SWI_annotation_old", patient_id, osp.basename(nii_path)))
# total 452, exclude 5 cases where there are more than 1 nii files (457->452)

In [11]:
# CRCKS annotation (0,1,2) => AISCAN annotation (0,1) : Set all 'possible CMBs' to 'Non-CMB'
exist = set()
for nii_path in tqdm(glob("/data000/jhpark/CMB/SWI_annotation/*/*")):
    folder = osp.dirname(nii_path)
    patient_id = osp.basename(folder)
    if patient_id in exist:
        print(patient_id, "has more than one nii file. Skip it") # ["CA-01419-1", "HS-09266-1", "YN-10087-1", "HS-09262-1", "HS-09253-1"]
        continue
    exist.add(patient_id)
    nii_array = np.array(nib.load(nii_path).dataobj)
    nii_array[nii_array == 2] = 0 # possible CMB => Non-CMB
    if nii_array.sum() == 0:
        print(patient_id, " has no CMBs now")
        continue
    os.makedirs(osp.join("/data000/mhchoi/CMB/SWI_annotation_old_exclude", patient_id), exist_ok=True)
    nib.save(nib.Nifti1Image(nii_array, np.eye(4)), osp.join("/data000/mhchoi/CMB/SWI_annotation_old_exclude", patient_id, osp.basename(nii_path)))

 24%|██▍       | 111/457 [02:16<06:37,  1.15s/it]

CA-01419-1 has more than one nii file. Skip it


 56%|█████▌    | 255/457 [04:59<04:50,  1.44s/it]

HS-09266-1 has more than one nii file. Skip it


 74%|███████▎  | 336/457 [06:26<01:59,  1.02it/s]

...

 88%|████████▊ | 402/457 [07:38<01:17,  1.42s/it]

HS-09262-1 has more than one nii file. Skip it


 89%|████████▉ | 406/457 [07:41<00:42,  1.20it/s]

HS-09253-1 has more than one nii file. Skip it


100%|██████████| 457/457 [08:48<00:00,  1.16s/it]


In [ ]:
# CRCKS annotation (0,1,2) => AISCAN annotation (0,1) : Leave only possible CMB
exist = set()
for nii_path in tqdm(glob("/data000/jhpark/CMB/SWI_annotation/*/*")):
    folder = osp.dirname(nii_path)
    patient_id = osp.basename(folder)
    if patient_id in exist:
        print(patient_id, "has more than one nii file. Skip it") # ["CA-01419-1", "HS-09266-1", "YN-10087-1", "HS-09262-1", "HS-09253-1"]
        continue
    exist.add(patient_id)
    nii_array = np.array(nib.load(nii_path).dataobj)
    nii_array[nii_array == 1] = 0 # CMB => non-CMB
    nii_array[nii_array == 2] = 1 # possible CMB => CMB
    if nii_array.sum() == 0:
        print(patient_id, " has no CMBs now")
        continue
    os.makedirs(osp.join("/data000/mhchoi/CMB/SWI_annotation_old_only", patient_id), exist_ok=True)
    nib.save(nib.Nifti1Image(nii_array, np.eye(4)), osp.join("/data000/mhchoi/CMB/SWI_annotation_old_only", patient_id, osp.basename(nii_path)))

In [ ]:
# Number of series of old/new data
old_hospitals = defaultdict(int)
new_hospitals = defaultdict(int)
for path in glob("/data000/mhchoi/CMB/SWI_annotation_new/*"):
    hospital = osp.basename(path).split("-")[0]
    old_hospitals[hospital] += 1

for path in glob("/data000/mhchoi/CMB/SWI_annotation_old/*"):
    hospital = osp.basename(path).split("-")[0]
    new_hospitals[hospital] += 1
    
for hospital in set(list(new_hospitals.keys()) + list(old_hospitals.keys())):
    print(f"{HOSPITALS[hospital]} / pre : {new_hospitals[hospital]}, post : {old_hospitals[hospital]}, sum : {new_hospitals[hospital] + old_hospitals[hospital]}")
    
# Total 449, exclude 3 corrupted cases (452->449)

HUMC (한림대학교병원) / pre : 78, post : 260, sum : 338
DAMC (동아대학교병원) / pre : 2, post : 0, sum : 2
seoulMC (서울의료원) / pre : 24, post : 132, sum : 156
JNUH (제주대학교병원) / pre : 71, post : 56, sum : 127
DSMC (계명대학교병원) / pre : 20, post : 227, sum : 247
YUMC (영남대학교병원) / pre : 12, post : 0, sum : 12
DUMC (동국대학교병원) / pre : 45, post : 0, sum : 45
CAUMC (중앙대학교병원) / pre : 37, post : 0, sum : 37
SCHMC (순천향대학교병원) / pre : 30, post : 0, sum : 30
KUMC (고려대학교병원) / pre : 0, post : 36, sum : 36
DEMC (동의의료원) / pre : 64, post : 0, sum : 64
CBNUH (충북대학교병원) / pre : 69, post : 0, sum : 69


In [48]:
# count cmb in each series of old(no exclude) / old(exclude) / new data
cmb_count_old = {}
cmb_count_old_exclude = {}
cmb_count_new = {}

for nii_path in tqdm(glob("/data000/mhchoi/CMB/SWI_annotation_old/*/*")):
    patient_id = osp.basename(osp.dirname(nii_path))
    cmbs = cmb_count(np.array(nib.load(nii_path).dataobj))
    cmb_count_old[patient_id] = cmbs

with open("metadata/R1/cmb_count_old.json", "w") as f:
    json.dump(cmb_count_old, f)

for nii_path in tqdm(glob("/data000/mhchoi/CMB/SWI_annotation_old_exclude/*/*")):
    patient_id = osp.basename(osp.dirname(nii_path))
    cmbs = cmb_count(np.array(nib.load(nii_path).dataobj))
    cmb_count_old_exclude[patient_id] = cmbs

with open("metadata/R1/cmb_count_old_exclude.json", "w") as f:
    json.dump(cmb_count_old_exclude, f)

for nii_path in tqdm(glob("/data000/mhchoi/CMB/SWI_annotation_new/*/*")):
    patient_id = osp.basename(osp.dirname(nii_path))
    cmbs = cmb_count(np.array(nib.load(nii_path).dataobj))
    cmb_count_new[patient_id] = cmbs

with open("metadata/R1/cmb_count_new.json", "w") as f:
    json.dump(cmb_count_new, f)

100%|██████████| 711/711 [11:26<00:00,  1.04it/s]


In [8]:
# minimum / maximum cmbs of three sets

with open("metadata/R1/cmb_count_old.json", "r") as f:
    cmb_count_old_list = list(json.load(f).values())
with open("metadata/R1/cmb_count_old_exclude.json", "r") as f:
    cmb_count_old_exclude_list = list(json.load(f).values())
with open("metadata/R1/cmb_count_new.json", "r") as f:
    cmb_count_new_list = list(json.load(f).values())
print(min(cmb_count_old_list), max(cmb_count_old_list))
print(min(cmb_count_old_exclude_list), max(cmb_count_old_exclude_list))
print(min(cmb_count_new_list), max(cmb_count_new_list))

1 27
1 27
1 184


In [ ]:
# Create a figure
plt.figure(figsize=(8, 6))

# Plot overlapping histograms with transparency
plt.hist(cmb_count_old_list, bins=range(1, 28), color='blue', alpha=0.5, edgecolor='black', density=True)


# Add labels and legend
plt.xlabel("Count")
plt.ylabel("Normalized frequency")
plt.title("Possible CMB → CMB")

# Show plot
plt.show()


In [ ]:
# Create a figure
plt.figure(figsize=(8, 6))

# Plot overlapping histograms with transparency
plt.hist(cmb_count_old_exclude_list, bins=range(1, 28), color='red', alpha=0.5, edgecolor='black', density=True)


# Add labels and legend
plt.xlabel("Count")
plt.ylabel("Normalized frequency")
plt.title("Possible CMB → Non-CMB")

# Show plot
plt.show()

In [ ]:
# Create a figure
plt.figure(figsize=(8, 6))

# Plot overlapping histograms with transparency
plt.hist(cmb_count_old_list, bins=range(1, 28), color='blue', alpha=0.5, edgecolor='black', density=True)

# Add labels and legend
plt.xlabel("Count")
plt.ylabel("Normalized frequency")
plt.title("old dataset")
# Show plot
plt.show()


In [ ]:
# Create a figure
plt.figure(figsize=(8, 6))

# Plot overlapping histograms with transparency
plt.hist(cmb_count_new_list, bins=range(1, 28), color='red', alpha=0.5, edgecolor='black', density=True)

# Add labels and legend
plt.xlabel("Count")
plt.ylabel("Normalized frequency")
plt.title("new dataset")

# Show plot
plt.show()


In [17]:
anno_path_new = '/data000/mhchoi/CMB/SWI_annotation_new'
anno_path_old = '/data000/mhchoi/CMB/SWI_annotation_old'

nii_list_new = glob(osp.join(anno_path_new, "*/*.nii"))
nii_list_old = glob(osp.join(anno_path_old, "*/*.nii"))

old_data_paths = []
for i in range(1, 6):
    old_data_paths += glob(f"/data000/mjryu/SWI_crcsk/SWI_{i}/*")
old_data_path_map = {osp.basename(p) : p for p in old_data_paths}
print(old_data_path_map)

{'CB-05434-1': '/data000/mjryu/SWI_crcsk/SWI_1/CB-05434-1', 'GN-05220-1': '/data000/mjryu/SWI_crcsk/SWI_1/GN-05220-1', 'YN-10032-1': '/data000/mjryu/SWI_crcsk/SWI_1/YN-10032-1', 'SM-03379-1': '/data000/mjryu/SWI_crcsk/SWI_1/SM-03379-1', 'YN-10045-1': '/data000/mjryu/SWI_crcsk/SWI_1/YN-10045-1', 'CB-05408-1': '/data000/mjryu/SWI_crcsk/SWI_1/CB-05408-1', 'GN-05212-1': '/data000/mjryu/SWI_crcsk/SWI_1/GN-05212-1', 'DE-06967-1': '/data000/mjryu/SWI_crcsk/SWI_1/DE-06967-1', 'SH-04869-1': '/data000/mjryu/SWI_crcsk/SWI_1/SH-04869-1', 'GN-05255-1': '/data000/mjryu/SWI_crcsk/SWI_1/GN-05255-1', 'DE-06956-1': '/data000/mjryu/SWI_crcsk/SWI_1/DE-06956-1', 'KMD-09275-1': '/data000/mjryu/SWI_crcsk/SWI_1/KMD-09275-1', 'CB-05451-1': '/data000/mjryu/SWI_crcsk/SWI_1/CB-05451-1', 'SH-04843-1': '/data000/mjryu/SWI_crcsk/SWI_1/SH-04843-1', 'KMD-09274-1': '/data000/mjryu/SWI_crcsk/SWI_1/KMD-09274-1', 'SM-03404-1': '/data000/mjryu/SWI_crcsk/SWI_1/SM-03404-1', 'SM-03433-1': '/data000/mjryu/SWI_crcsk/SWI_1/SM-03

In [13]:
# check if there's an intersection
old_set = set(map(lambda k:osp.basename(k), glob("/data000/mhchoi/CMB/SWI_annotation_new/*")))
new_set = set(map(lambda k:osp.basename(k), glob("/data000/mhchoi/CMB/SWI_annotation_old/*")))
intersection = list(old_set.intersection(new_set))
dup_list = []

not_match = set()
for patient_id in tqdm(intersection):
	old_nii_path = glob(osp.join("/data000/mhchoi/CMB/SWI_annotation_old", patient_id, "*"))[0]
	new_nii_path = glob(osp.join("/data000/mhchoi/CMB/SWI_annotation_new", patient_id, "*"))[0]
	old_uid = osp.basename(old_nii_path)[:-4]
	new_uid = osp.basename(new_nii_path)[:-4]
	old_array = np.array(nib.load(old_nii_path).dataobj)
	new_array = np.array(nib.load(new_nii_path).dataobj)
	if np.array_equal(old_array.shape, new_array.shape) or old_uid == new_uid :
		dup_list.append(patient_id)

dup_list = intersection

print(dup_list)
print(len(dup_list))

100%|██████████| 64/64 [00:26<00:00,  2.39it/s]

['HS-09377-1', 'HS-09309-1', 'HS-09442-1', 'GN-05283-1', 'KMD-09240-1', 'HS-09398-1', 'HS-09342-1', 'HS-09435-1', 'HS-09336-1', 'HS-09426-1', 'GN-05340-1', 'HS-05237-2', 'HS-09410-1', 'KMD-09263-1', 'KMD-09229-1', 'KMD-09268-1', 'HS-09308-1', 'HS-09329-1', 'KMD-09245-1', 'KMD-09254-1', 'GN-05268-1', 'HS-09378-1', 'HS-09256-2', 'HS-09411-1', 'GN-03784-2', 'HS-09368-1', 'HS-09351-1', 'HS-09422-1', 'KMD-09252-1', 'GN-05318-1', 'HS-09446-1', 'HS-09429-1', 'HS-09416-1', 'HS-09252-1', 'HS-09284-1', 'HS-09266-1', 'KMD-09255-1', 'KMD-09242-1', 'HS-09267-1', 'HS-09421-1', 'HS-09318-1', 'HS-09379-2', 'HS-09344-1', 'GN-05272-1', 'GN-05311-1', 'HS-09276-1', 'KMD-09232-1', 'HS-09349-1', 'HS-09438-1', 'HS-09393-1', 'HS-09431-1', 'HS-09413-1', 'HS-09335-1', 'HS-09296-1', 'HS-09436-1', 'HS-09339-1', 'HS-09253-1', 'HS-09400-1', 'GN-05320-1', 'KMD-09246-1', 'HS-09363-1', 'HS-09355-1', 'KMD-09248-1', 'GN-05258-1']
64


In [9]:
corrupted = ["KMD-09232-1", "KMD-09228-1", "SM-03376-1"]
# with open("SWI_annotation_metadata/R1/exclude.json", "w") as f:
#     json.dump({"corrupted" : corrupted, "duplicate" : dup_list}, f)

In [ ]:
SWI_path_sum_old = []
SWI_path_sum_new = []

############### CRCSK ###############
no_matching = []
exist_uid_old = set()
exist_uid_new = set()
print("CRCSK extraction start")
for nii_path in tqdm(nii_list_old):
    patient_id = osp.basename(osp.dirname(nii_path))
    if patient_id in corrupted or patient_id in dup_list: continue
    nii_uid = osp.basename(nii_path)[:-4]
    if nii_uid in exist_uid_old: 
        print(f"{patient_id} has exist uid")
    else: 
        exist_uid_old.add(nii_uid)
    matching_series = None
    for series in glob(osp.join(old_data_path_map[patient_id], "*")):
        
        dcm_list = os.listdir(series)
        dcm_uid = pydicom.dcmread(osp.join(series, dcm_list[0]), force=True).SeriesInstanceUID
        if nii_uid == dcm_uid:
            matching_series = series
            break
    if matching_series is None:
        print(old_data_path_map[patient_id], "has no matching series! Skip")
        continue
    dcm_sample_paths = os.listdir(matching_series)
    dcm_sample = pydicom.dcmread(osp.join(matching_series, dcm_sample_paths[0]), force=True)

    SWI_path_sum_old.append([patient_id, 
                         dcm_sample.SeriesDescription, 
                         matching_series, # data path
                         nii_path, # GT path
                         dcm_sample.Manufacturer,
                         len(dcm_sample_paths)
                         ])

nii_check = defaultdict(list)

############### AISCAN ###############
aiscan_data_root = "/data000/jhpark/CMB/SWI_data_AISCAN"
print("AISCAN extraction start")
for nii_path in tqdm(nii_list_new):
    patient_id = osp.basename(osp.dirname(nii_path))cd 
    hospital_id = HOSPITALS_ENG[patient_id.split("-")[0]]
    nii_uid = osp.basename(nii_path)[:-4]
    nii_check[nii_uid].append(patient_id)
    if nii_uid in exist_uid_new: 
        print(f"{patient_id} has exist uid")
    else: 
        exist_uid_new.add(nii_uid)
        
    matching_series = None
    for series in glob(osp.join(aiscan_data_root, hospital_id, patient_id, "*/*/SWI")):
        dcm_list = os.listdir(series)
        if len(dcm_list) == 0: # T1 images
            continue
        dcm_uid = pydicom.dcmread(osp.join(series, dcm_list[0]), force=True).SeriesInstanceUID
        if nii_uid == dcm_uid:
            matching_series = series
            break
    if matching_series is None:
        print(f"{patient_id} has no matching series! Skip")
        continue

    dcm_sample_paths = os.listdir(osp.join(matching_series))
    dcm_sample = pydicom.dcmread(osp.join(matching_series, dcm_sample_paths[0]), force=True)

    SWI_path_sum_new.append([patient_id, 
                         dcm_sample.SeriesDescription, 
                         matching_series, # data path
                         nii_path, # GT Path
                         dcm_sample.Manufacturer,
                         len(dcm_sample_paths)
                         ])
    
# with open("SWI_annotation_metadata/R1/SWI_path_sum_old.json", "w") as f:
#     json.dump(sorted(SWI_path_sum_old, key=lambda k:k[0]), f)
# with open("SWI_annotation_metadata/R1/SWI_path_sum_new.json", "w") as f:
#     json.dump(sorted(SWI_path_sum_new, key=lambda k:k[0]), f)
# with open("SWI_annotation_metadata/R1/SWI_path_sum_merge.json", "w") as f:
#     json.dump(sorted(SWI_path_sum_old + SWI_path_sum_new, key=lambda k:k[0]), f)

CRCSK extraction start


  0%|          | 0/452 [00:00<?, ?it/s]

100%|██████████| 452/452 [00:07<00:00, 62.20it/s]


AISCAN extraction start


 38%|███▊      | 268/711 [00:03<00:07, 56.29it/s]

...

 66%|██████▌   | 470/711 [00:06<00:03, 73.00it/s]

KMD-09450-2 has exist uid


 96%|█████████▋| 686/711 [00:09<00:00, 74.84it/s]

KMD-09240-1 has exist uid
SM-04202-1 has exist uid


100%|██████████| 711/711 [00:09<00:00, 71.49it/s]


In [20]:
[v for k, v in nii_check.items() if len(v) > 1]

[['SM-04208-1', 'SM-04202-1'],
 ['SM-04099-1', 'SM-04051-1'],
 ['KMD-09450-1', 'KMD-09450-2'],
 ['SM-04145-1', 'SM-04145-2'],
 ['KMD-09431-1', 'KMD-09431-2'],
 ['KMD-09240-2', 'KMD-09240-1']]

In [3]:
with open("SWI_annotation_metadata/R1/SWI_path_sum_merge.json", "r") as f:
    SWI_path_sum = json.load(f)

SWAN_count = 0
minIP_count = 0
MAG_count = 0
SWI_count = 0
minIP_hospitals = defaultdict(int)
SWAN_hospitals = defaultdict(int)
MAG_hospitals = defaultdict(int)
SWI_hospitals = defaultdict(int)
variant = {"SWAN" : [], "minIP" : [], 'MAG' : [], "SWI": []}
for code, description, _, _, manufacturer, _ in SWI_path_sum:
    hospital = code.split("-")[0]
    if 'SWAN' in description:
        SWAN_count+=1
        SWAN_hospitals[hospital] += 1
        variant["SWAN"].append(code)
        continue
    if 'minIP' in description:
        minIP_count+=1
        minIP_hospitals[hospital] += 1
        variant["minIP"].append(code)
        continue
    if 'Mag' in description or 'sSWIp_M' in description:
        MAG_count+=1
        MAG_hospitals[hospital] += 1
        variant["MAG"].append(code)
        continue
    if 'SWI' in description:
        SWI_count+=1
        SWI_hospitals[hospital] += 1
        variant["SWI"].append(code)
    else:
        print(description)
        raise NotImplementedError
        
print("SWAN :", SWAN_count)
print("minIP :", minIP_count)
print("MAG :", MAG_count)
print("SWI : ", SWI_count)

print("SWAN hospitals :", SWAN_hospitals)
print("SWI hopsitals:", SWI_hospitals)
print("minIP hospitals :", minIP_hospitals)
print("MAG hospitals :", MAG_hospitals)
print("SUM : ", SWAN_count + MAG_count + SWI_count + minIP_count)
with open("SWI_annotation_metadata/R1/variant.json", "w") as f:
    json.dump(variant, f)

SWAN : 62
minIP : 15
MAG : 186
SWI :  828
SWAN hospitals : defaultdict(<class 'int'>, {'DA': 2, 'KMD': 30, 'SH': 30})
SWI hopsitals: defaultdict(<class 'int'>, {'CA': 34, 'CB': 2, 'DE': 64, 'DI': 45, 'GN': 112, 'HS': 285, 'KG': 11, 'KMD': 130, 'SM': 137, 'YN': 8})
minIP hospitals : defaultdict(<class 'int'>, {'SM': 15})
MAG hospitals : defaultdict(<class 'int'>, {'CA': 3, 'CB': 67, 'GN': 6, 'HS': 10, 'KG': 25, 'KMD': 71, 'YN': 4})
SUM :  1091


In [3]:
with open("SWI_annotation_metadata/R2/SWI_path_sum.json", "r") as f:
    SWI_path_sum = json.load(f)

siemens = 0
ge = 0
philips = 0
others = 0
for code, description, _, _, manufacturer, _ in SWI_path_sum:
    if "siemens" in manufacturer.lower():
        siemens += 1
    elif "philips" in manufacturer.lower():
        philips += 1
    elif "ge" in manufacturer.lower():
        ge += 1
    else:
        others += 1
print("SIEMENS", siemens)
print("GE", ge)
print("Philips", philips)
print("Others", others)
print("SUM : ", siemens+ge+philips+others)

SIEMENS 472
GE 32
Philips 506
Others 0
SUM :  1010


In [75]:
with open("metadata/R1/SWI_path_sum_merge.json", "r") as f:
    SWI_path_sum = json.load(f)
merge_hospitals = defaultdict(int)

for code, *_ in SWI_path_sum:
    hospital = code.split("-")[0]
    merge_hospitals[hospital] += 1
for hospital, count in merge_hospitals.items():
    print(f"{HOSPITALS[hospital]} : {count}")
print("SUM :", sum(merge_hospitals.values()))

CAUMC (중앙대학교병원) : 37
CBNUH (충북대학교병원) : 69
DAMC (동아대학교병원) : 2
DEMC (동의의료원) : 64
DUMC (동국대학교병원) : 45
JNUH (제주대학교병원) : 118
HUMC (한림대학교병원) : 295
KUMC (고려대학교병원) : 36
DSMC (계명대학교병원) : 231
SCHMC (순천향대학교병원) : 30
seoulMC (서울의료원) : 152
YUMC (영남대학교병원) : 12
SUM : 1091


In [9]:
with open("SWI_annotation_metadata/R1/SWI_path_sum_merge.json", "r") as f:
    SWI_path_sum = json.load(f)
with open("SWI_annotation_metadata/R1/variant.json", "r") as f:
    variants = json.load(f)

all_train = []
all_test = []

############### SWAN ###############

SWAN = variants["SWAN"]
hospital_list = defaultdict(list)

for code in SWAN:
    hospital = code.split("-")[0]
    hospital_list[hospital].append(code)

for codes in hospital_list.values():
    all_train.extend(codes[:int(len(codes) * 0.8)])
    all_test.extend(codes[int(len(codes) * 0.8):])

############### MAG ###############

MAG = variants["MAG"]
hospital_list = defaultdict(list)

for code in MAG:
    hospital = code.split("-")[0]
    hospital_list[hospital].append(code)

for codes in hospital_list.values():
    all_train.extend(codes[:int(len(codes) * 0.8)])
    all_test.extend(codes[int(len(codes) * 0.8):])

############### SWI ###############

swi = variants["SWI"]
hospital_list = defaultdict(list)

for code in swi:
    hospital = code.split("-")[0]
    hospital_list[hospital].append(code)

for codes in hospital_list.values():
    all_train.extend(codes[:int(len(codes) * 0.8)])
    all_test.extend(codes[int(len(codes) * 0.8):])

with open("SWI_annotation_metadata/R1/train_data.json", "w") as f:
    json.dump(all_train, f)
with open("SWI_annotation_metadata/R1/test_data.json", "w") as f:
    json.dump(all_test, f)

In [12]:
with open("SWI_annotation_metadata/R1/train_data.json", "r") as f:
    all_train = json.load(f)
with open("SWI_annotation_metadata/R1/test_data.json", "r") as f:
    all_test = json.load(f)
print("Training data institution analysis")
train_hospitals = defaultdict(int)
test_hospitals = defaultdict(int)
for code in all_train:
    hospital = code.split("-")[0]
    train_hospitals[hospital] += 1
for hospital, count in train_hospitals.items():
    print(f"{HOSPITALS[hospital]} : {count}")
print(f"SUM : {sum(train_hospitals.values())}")
print("\nTest data institution analysis")
for code in all_test:
    hospital = code.split("-")[0]
    test_hospitals[hospital] += 1
for hospital, count in test_hospitals.items():
    print(f"{HOSPITALS[hospital]} : {count}")
print(f"SUM : {sum(test_hospitals.values())}")

Training data institution analysis
DAMC (동아대학교병원) : 1
DSMC (계명대학교병원) : 184
SCHMC (순천향대학교병원) : 24
CAUMC (중앙대학교병원) : 29
CBNUH (충북대학교병원) : 54
JNUH (제주대학교병원) : 93
HUMC (한림대학교병원) : 236
KUMC (고려대학교병원) : 28
YUMC (영남대학교병원) : 9
DEMC (동의의료원) : 51
DUMC (동국대학교병원) : 36
seoulMC (서울의료원) : 109
SUM : 854

Test data institution analysis
DAMC (동아대학교병원) : 1
DSMC (계명대학교병원) : 47
SCHMC (순천향대학교병원) : 6
CAUMC (중앙대학교병원) : 8
CBNUH (충북대학교병원) : 15
JNUH (제주대학교병원) : 25
HUMC (한림대학교병원) : 59
KUMC (고려대학교병원) : 8
YUMC (영남대학교병원) : 3
DEMC (동의의료원) : 13
DUMC (동국대학교병원) : 9
seoulMC (서울의료원) : 28
SUM : 222


In [14]:
with open("SWI_annotation_metadata/R1/SWI_path_sum_merge.json", "r") as f:
    SWI_path_sum = json.load(f)
    code_to_manu = {code:manu for code, _, _, _, manu, _ in SWI_path_sum}
with open("SWI_annotation_metadata/R1/train_data.json", "r") as f:
    all_train = json.load(f)
with open("SWI_annotation_metadata/R1/test_data.json", "r") as f:
    all_test = json.load(f)
siemens = 0
ge = 0
philips = 0
others = 0
print("Training data manufacturer analysis")
for code in all_train:
    manufacturer = code_to_manu[code]
    if "siemens" in manufacturer.lower():
        siemens += 1
    elif "philips" in manufacturer.lower():
        philips += 1
    elif "ge" in manufacturer.lower():
        ge += 1
    else:
        others += 1
print("SIEMENS", siemens)
print("GE", ge)
print("Philips", philips)
print("Others", others)
print("SUM : ", siemens+ge+philips+others)
    
siemens = 0
ge = 0
philips = 0
others = 0
print("\nTest data manufacturer analysis")
for code in all_test:
    manufacturer = code_to_manu[code]
    if "siemens" in manufacturer.lower():
        siemens += 1
    elif "philips" in manufacturer.lower():
        philips += 1
    elif "ge" in manufacturer.lower():
        ge += 1
    else:
        others += 1
print("SIEMENS", siemens)
print("GE", ge)
print("Philips", philips)
print("Others", others)
print("SUM : ", siemens+ge+philips+others)

Training data manufacturer analysis
SIEMENS 379
GE 49
Philips 426
Others 0
SUM :  854

Test data manufacturer analysis
SIEMENS 104
GE 13
Philips 105
Others 0
SUM :  222


In [16]:
with open("SWI_annotation_metadata/R1/train_data.json", "r") as f:
    train_data = json.load(f)
with open("SWI_annotation_metadata/R1/test_data.json", "r") as f:
    test_data = json.load(f)

included = set(train_data + test_data)

with open("SWI_annotation_metadata/R1/SWI_path_sum_merge.json", "r") as f:
    SWI_path_sum = json.load(f)

SWI_path_sum_update = []
for item in SWI_path_sum:
    if item[0] in included:
        SWI_path_sum_update.append(item)
with open("SWI_annotation_metadata/R1/SWI_path_sum_merge_filtered.json", "w") as f:
    json.dump(SWI_path_sum_update, f)

In [ ]:
with open("SWI_annotation_metadata/R1/SWI_path_sum_merge_filtered.json", "r") as f:
    SWI_path_sum = json.load(f)

base_path = '/data000/mhchoi/CMB'
Task_ID = 'Task102_CMB_SWI'

os.makedirs(os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'imagesTr'), exist_ok=True) ### training
os.makedirs(os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'imagesTs'), exist_ok=True) ### test
os.makedirs(os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'labelsTr'), exist_ok=True)
os.makedirs(os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'labelsTs'), exist_ok=True)

imagesTr_dir = os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'imagesTr')
imagesTs_dir = os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'imagesTs')
labelsTr_dir = os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'labelsTr')
labelsTs_dir = os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'labelsTs')

base_path = r'/data000/mhchoi/CMB'

NP = 50 # number of processes
LIST = SWI_path_sum
NPC = math.ceil(len(LIST)/NP)
T = [LIST[NPC*i:NPC*(i+1)] for i in range(NP)] # slice it for multiprocessing

def func(process_i):
    SWI_ID_pair = []
    corrupted = []
    count_num = 1 + NPC * process_i
    for i, (code, description, dcm_path, gt_path, manufacturer, _) in enumerate(T[process_i]):
        gt_array = np.array(nib.load(gt_path).dataobj)
        assert np.sum(gt_array) > 0, "No CMB data must be excluded in prior, this should not happen!"
        dcm_list = [j for j in os.listdir(dcm_path)]
        dcm_list.sort()
        
        dcm_sitk = sitk.ReadImage(os.path.join(dcm_path,dcm_list[0]))
        dcm_info = pydicom.dcmread(os.path.join(dcm_path,dcm_list[0]),force=True)
        if hasattr(dcm_info,'SpacingBetweenSlices'):
            sp_info = [dcm_info.PixelSpacing[0],dcm_info.PixelSpacing[1],dcm_info.SpacingBetweenSlices]
        elif hasattr(dcm_info,'SliceThickness'):
            sp_info = [dcm_info.PixelSpacing[0],dcm_info.PixelSpacing[1],dcm_info.SliceThickness]
        else:
            dcm_info2 = pydicom.dcmread(os.path.join(dcm_path,dcm_list[1]),force=True)
            sl_value = np.abs(np.float32(dcm_info.ImagePositionPatient[2])-np.float32(dcm_info2.ImagePositionPatient[2]))
            sp_info = [dcm_info.PixelSpacing[0],dcm_info.PixelSpacing[1], sl_value]
            
        d_info_tmp = dcm_sitk.GetDirection()
        d_info_tmp = np.reshape(d_info_tmp,(3,3))
        d_info = np.ones((4,4))
        d_info[:3,:3] = d_info_tmp.copy()
        d_info[3,:3] = 0
        d_info[0,3] = 127.
        d_info[1,3] = -91
        d_info[2,3] = 73
        
        dcm_array = []
        sl_loc = []
        instance_num = []
        corrupt = False
        for sl in range(len(dcm_list)):
            dcm_info = pydicom.dcmread(os.path.join(dcm_path,dcm_list[sl]),force=True)
            dcm_info.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
            try:
                dcm_array.append(dcm_info.pixel_array)
            except (ValueError, AttributeError):
                print(f"It seems {code} is corrupted. Skip!")
                corrupted.append([code, dcm_path])
                corrupt = True
                break
            sl_loc.append(dcm_info.ImagePositionPatient[2])
            instance_num.append(dcm_info.InstanceNumber)
        if corrupt: continue

        dcm_array  = np.array(dcm_array)
        sort_idx = np.argsort(instance_num)
        dcm_array = np.array(dcm_array)[sort_idx]
        dcm_array = np.transpose(dcm_array,(1,2,0))
        
        image = nib.Nifti1Image(dcm_array,d_info)
        header_info = image.header
        header_info['pixdim'][1:4]  = sp_info
        image = nib.Nifti1Image(dcm_array, d_info, header_info)
        label = nib.Nifti1Image(gt_array, d_info, header_info)

        Inst_ID = code.split('-')[0]
        if len(Inst_ID)!=3:
            Inst_ID = 'S'+Inst_ID
            
        save_ID = Inst_ID+'_%04d.nii.gz' % count_num
        SWI_ID_pair.append([code, save_ID])
        nib.save(image, osp.join(imagesTr_dir, save_ID))
        nib.save(label, osp.join(labelsTr_dir, save_ID))
        count_num += 1
    return SWI_ID_pair, corrupted


with Pool(processes=NP) as pool:
    results = pool.map(func, range(0, NP))

with open(os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID, "SWI_ID_pair.json"), "w") as f:
    json.dump(sum(list(map(lambda k:k[0],results)), []), f)
with open(os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID, "corrupted.json"), "w") as f:
    json.dump(sum(list(map(lambda k:k[1],results)), []), f)

train_list = os.listdir(labelsTr_dir)
train_list.sort()

NP = 50 # number of processes
LIST = train_list
NPC = math.ceil(len(LIST)/NP)
T = [LIST[NPC*i:NPC*(i+1)] for i in range(NP)] # slice it for multiprocessing

def func(process_i):
    for save_ID in T[process_i]:
        for i, directory in enumerate([imagesTr_dir, labelsTr_dir]):
            nii_data = nib.load(os.path.join(directory,save_ID))
            header_info = nii_data.header
            data = np.array(nii_data.dataobj)
            if i == 1: data = np.uint8(data) # uint8 conversion for label
            d_info = header_info.get_base_affine()
            d_info[0,3] = 127.
            d_info[1,3] = -91
            d_info[2,3] = 73
            a= nib.Nifti1Image(data,d_info,header_info)
            nib.save(a,os.path.join(directory,save_ID))

with Pool(processes=NP) as pool:
    pool.map(func, range(0, NP))

In [ ]:
with open("SWI_annotation_metadata/R1/test_data.json", "r") as f:
    all_test = set(json.load(f))
for code, save_ID in tqdm(json.load(open(osp.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID, "SWI_ID_pair.json"), 'r'))):
    if code in all_test:
        os.rename(osp.join(osp.join(imagesTr_dir, save_ID)), osp.join(osp.join(imagesTs_dir, save_ID)))
        os.rename(osp.join(osp.join(labelsTr_dir, save_ID)), osp.join(osp.join(labelsTs_dir, save_ID)))

100%|██████████| 1076/1076 [00:01<00:00, 698.68it/s]


In [93]:
imagesTr_dir = os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'imagesTr')
imagesTs_dir = os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID,'imagesTs')

def get_identifiers_from_splitted_files(folder: str):
    uniques = np.unique([i.split('.')[0] for i in os.listdir(folder) if i.endswith('.nii.gz')])
    return uniques

dataset_name = 'Merged'
dataset_description = 'SWI image based CMB segmentation dataset'
dataset_reference  = 'None'
license = 'None'
dataset_release = "1.0 02/07/2025"
modalities = ['SWI']
dataset_modality = {str(i): modalities[i] for i in range(len(modalities))}
labels = ['Non CMB','CMB']
dataset_labels = {str(i): labels[i] for i in range(len(labels))}
train_identifiers = get_identifiers_from_splitted_files(imagesTr_dir)
test_identifiers = get_identifiers_from_splitted_files(imagesTs_dir)

json_dict = {}
json_dict['name'] = dataset_name
json_dict['description'] = dataset_description
json_dict['tensorImageSize'] = "5D"
json_dict['reference'] = dataset_reference
json_dict['licence'] = license
json_dict['release'] = dataset_release
json_dict['modality'] = dataset_modality
json_dict['labels'] = dataset_labels

json_dict['numTraining'] = len(train_identifiers)
json_dict['numTest'] = len(test_identifiers)
json_dict['training'] = [
    {'image': "./imagesTr/%s.nii.gz" % i, "label": "./labelsTr/%s.nii.gz" % i} for i in train_identifiers]
json_dict['test'] = ["./imagesTs/%s.nii.gz" % i for i in test_identifiers]

import json
Task_ID = 'Task102_CMB_SWI'
json_text = json.dumps(json_dict)
save_path = os.path.join(base_path,'nnUNet_raw_data_base','nnUNet_raw_data',Task_ID)
json_save_path = os.path.join(save_path, 'dataset.json')
with open(json_save_path,'w') as fp:
    fp.write(json_text)
    
print('done')

done


In [1]:
import json
with open("SWI_annotation_metadata/R2_external/SWI_path_sum.json", "r") as f:
    SWI_path_sum = json.load(f)

siemens = 0
ge = 0
philips = 0
others = 0
for code, description, _, _, manufacturer, _ in SWI_path_sum:
    if "siemens" in manufacturer.lower():
        siemens += 1
    elif "philips" in manufacturer.lower():
        philips += 1
    elif "ge" in manufacturer.lower():
        ge += 1
    else:
        others += 1
print("SIEMENS", siemens)
print("GE", ge)
print("Philips", philips)
print("Others", others)
print("SUM : ", siemens+ge+philips+others)

SIEMENS 147
GE 83
Philips 60
Others 0
SUM :  290


In [1]:
import json
from collections import defaultdict
with open("SWI_annotation_metadata/R2_external/SWI_path_sum.json", "r") as f:
    SWI_path_sum = json.load(f)

SWAN_count = 0
minIP_count = 0
MAG_count = 0
SWI_count = 0
minIP_hospitals = defaultdict(int)
SWAN_hospitals = defaultdict(int)
MAG_hospitals = defaultdict(int)
SWI_hospitals = defaultdict(int)
variant = {"SWAN" : [], "minIP" : [], 'MAG' : [], "SWI": []}
for code, description, _, _, manufacturer, _ in SWI_path_sum:
    hospital = code.split("-")[0]
    if 'SWAN' in description:
        SWAN_count+=1
        SWAN_hospitals[hospital] += 1
        variant["SWAN"].append(code)
        continue
    if 'minIP' in description:
        minIP_count+=1
        minIP_hospitals[hospital] += 1
        variant["minIP"].append(code)
        continue
    if 'Mag' in description or 'sSWIp_M' in description:
        MAG_count+=1
        MAG_hospitals[hospital] += 1
        variant["MAG"].append(code)
        continue
    if 'SWI' in description:
        SWI_count+=1
        SWI_hospitals[hospital] += 1
        variant["SWI"].append(code)
    else:
        print(description)
        raise NotImplementedError
        
print("SWAN :", SWAN_count)
print("minIP :", minIP_count)
print("MAG :", MAG_count)
print("SWI : ", SWI_count)

print("SWAN hospitals :", SWAN_hospitals)
print("SWI hopsitals:", SWI_hospitals)
print("minIP hospitals :", minIP_hospitals)
print("MAG hospitals :", MAG_hospitals)
print("SUM : ", SWAN_count + MAG_count + SWI_count + minIP_count)

SWAN : 83
minIP : 0
MAG : 61
SWI :  146
SWAN hospitals : defaultdict(<class 'int'>, {'SH': 83})
SWI hopsitals: defaultdict(<class 'int'>, {'DI': 120, 'KG': 26})
minIP hospitals : defaultdict(<class 'int'>, {})
MAG hospitals : defaultdict(<class 'int'>, {'KG': 60, 'SH': 1})
SUM :  290
